In [ ]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
import numpy as np

import os
# This is to force CPU evaluation since we probably train on a bigger GPU than I have
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import keras

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Get shared constants and functions
%run "NN Audio Core.py"

In [ ]:
# Functions local to this notebook

# Sample output is (samples, bins) all converted to magnitude
def get_samples(file):
    wav, rate = librosa.core.load(file)
    samples = abs(get_ft(wav).T) # organized as bins, frames so we need to transpose them to frames, bins
    return samples

# Only need this for testing at this point?  Could still make it a 'top' type shared function
def clip_frames(file):
    samples = get_samples(file)
    frames = np.empty((samples.shape[0], WINDOW_SIZE, FFT_BINS, 1))
    half_win = WINDOW_SIZE//2
    padded_samples = np.concatenate([np.zeros((half_win, FFT_BINS)), samples, np.zeros((half_win, FFT_BINS))])
    for i in range(0, samples.shape[0]-1):
        frames[i,:,:,0] = padded_samples[i:i+WINDOW_SIZE,:]
    return frames

def display_fft(ft):
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(ft), ref=np.max), y_axis='log', x_axis='time')

def draw(wav):
    fft = get_ft(wav)
    display_fft(fft)
    return fft

In [ ]:
# some test data to hack around with
test_file = "Assets\\DataShareArchive\\Test\\Clean\\p232_010.wav"
wav, rate = librosa.core.load(test_file)
ft = get_ft(wav)

In [ ]:
del model  # Just to be sure, since sometimes the errors didn't seem to change much on reload
model = keras.models.load_model("best_model.h5")

In [ ]:
def diff_clip(wav_root, file):
    clean_dir = wav_root + "\\Clean\\"
    noisy_dir = wav_root + "\\Noisy\\"
    noisy_ft = get_ft_from_file(noisy_dir + file)
    clean_ft = get_ft_from_file(clean_dir + file)
    diff = diff_ft(clean_ft, noisy_ft)
    return diff


def clean_clip(model, n_file):
    verify_frames = clip_frames(n_file)
    output_targets = model.predict([verify_frames])
    wav, rate = librosa.core.load(n_file)
    n_fft = get_ft(wav)
    fft = rebuild_fft(output_targets, n_fft)
    return fft, inv_ft(fft)

# For now just skipping below minimum files, which means we may not get exact count of results
def compare_files(wav_root, model, n_files=50, min_origin_diff=20):
    clean_dir = wav_root + "\\Clean\\"
    noisy_dir = wav_root + "\\Noisy\\"
    file_list = os.listdir(clean_dir)
    file_index = 0
    count = min(n_files, len(file_list))
    diff_ratios = np.empty((count))
    while (file_index < count) :
        file = file_list[file_index]
        noisy_ft = get_ft_from_file(noisy_dir + file)
        clean_ft = get_ft_from_file(clean_dir + file)
        original_diff = diff_ft(clean_ft, noisy_ft)
        if (original_diff > min_origin_diff):
            model_ft, wav = clean_clip(model, noisy_dir + file)
            model_diff = diff_ft(model_ft, clean_ft)
            diff_ratios[file_index] = model_diff/original_diff
            print("%s  :  original %3.2f  :  ratio %.2f" %(file, original_diff, model_diff/original_diff))
        file_index += 1
    print("Average : %.2f" % np.average(diff_ratios))
    print("90%% pct %.2f" % np.percentile(diff_ratios, 90))
    return 

In [ ]:
compare_files("Assets\\DataShareArchive\\Test\\", model, 50)

In [ ]:
# Full round trip test
file = "p232_002.wav"

path = "Assets\\DataShareArchive\\Test\\"

verify_file = path + "Noisy\\" + file
clean_file = path + "Clean\\" + file

p_fft, p_wav = clean_clip(model, verify_file)

wav, rate = librosa.core.load(clean_file)
c_fft = get_ft(wav)

cleaned_diff = diff_ft(p_fft, c_fft)
original_diff = diff_clip(path, file)
ratio = cleaned_diff / original_diff

print("Cleaned diff  %.2f" % cleaned_diff)
print("Original diff %.2f" % original_diff)
print("Ratio         %.2f" % ratio)

print("Cleaned clip")

display_fft(p_fft)
Audio(p_wav,rate=22050)

In [ ]:
wav, rate = librosa.core.load(verify_file)
n_fft = draw(wav)

err_fft = c_fft - n_fft
print("Average abs err vs clean = ", np.mean(abs(err_fft)))

print("Noisy file")
Audio(wav, rate=rate)

In [ ]:
cut_fft = n_fft-p_fft
display_fft(cut_fft)

print("Removed audio")
print("Average cut value = ", np.mean(abs(cut_fft)))

cut_wav = inv_ft(cut_fft)
Audio(cut_wav,rate=22050)


In [ ]:
wav, rate = librosa.core.load(clean_file)
c_fft = draw(wav)

print("Clean sample")
Audio(wav, rate=rate)

Verification of ability to rebuild clean from noisy clip and perfect clean magnitudes

In [ ]:
# Round trip test with no NN evaluation to test pipeline
# Have to get phase information from the noisy file to match what happens for real

file = "p232_013.wav"
test_file = "Assets\\DataShareArchive\\Test\\Clean\\" + file
noisy_file = "Assets\\DataShareArchive\\Test\\Noisy\\" + file
wav, rate = librosa.core.load(test_file)
noisy_wav, rate = librosa.core.load(noisy_file)

noisy_ft = get_ft(noisy_wav)
samples = get_samples(test_file)

rt_ft = rebuild_fft(samples, noisy_ft)

rt_wav = inv_ft(rt_ft)
print(rt_ft.shape)

display_fft(rt_ft)
Audio(rt_wav,rate=22050)

In [ ]:
display_fft(get_ft(wav))
Audio(wav,rate=22050)